In [1]:
import pandas as pd
import numpy as np

In [2]:
movies=pd.read_csv('movies.csv')
ratings=pd.read_csv('ratings.csv')
links=pd.read_csv('links.csv')
tags=pd.read_csv('tags.csv')

### Create movie profile by genres. each columns is 0/1 indicator for each genre

In [3]:
movies2=movies['genres'].apply(lambda x: x.split('|'))

In [4]:
##Create a list of unique Movie Genres
unique_genres=list()
for x in range(len(movies2)):
    for y in range(len(movies2[x])):
        if movies2[x][y] not in unique_genres:
            unique_genres.append(movies2[x][y])

In [5]:
movies.head(2)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


In [6]:
movies3=pd.DataFrame(movies2)

In [7]:
movies3['genres'].head(2)

0    [Adventure, Animation, Children, Comedy, Fantasy]
1                       [Adventure, Children, Fantasy]
Name: genres, dtype: object

In [8]:
for z in unique_genres:
    movies3[z]=0

In [9]:
for x in range(len(movies)):
    for y in range(len(movies3.columns)):
        if str(movies3.columns[y]) in str(movies3.iloc[x,0]):
            movies3.iloc[x,y]=1

In [10]:
movies3.head(3)

,genres,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,"[Adventure, Animation, Children, Comedy, Fantasy]",1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"[Adventure, Children, Fantasy]",1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"[Comedy, Romance]",0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Use numpy to calculate the similarity matrix (m x m)
#### i) Normalize each row by norm (A/|A| etc)


In [11]:
movies3['Norm']=np.sqrt(np.count_nonzero(movies3.iloc[:,1:],axis=1))

In [12]:
movies3.head(3)

,genres,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,...,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed),Norm
0,"[Adventure, Animation, Children, Comedy, Fantasy]",1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.236068
1,"[Adventure, Children, Fantasy]",1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.732051
2,"[Comedy, Romance]",0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1.414214


In [13]:
movies4=movies3.iloc[:,1:]

In [14]:
for i in movies4.columns:
    movies4[i]=movies4[i]/movies4['Norm']

In [15]:
movies4=movies4.iloc[:,:-1]

### Use numpy to calculate the similarity matrix (m x m)
#### ii) Obtain similarity matrix by M dot Mt

In [16]:
similarity_matrix=np.dot(movies4, movies4.T)

In [17]:
similarity_matrix2=pd.DataFrame(similarity_matrix,columns=movies['movieId']) #Remeber: index is equal to genres

In [18]:
similarity_matrix2=pd.merge(similarity_matrix2,movies,left_on=similarity_matrix2.index,right_on=movies.index,how='inner')

In [19]:
similarity_matrix2=similarity_matrix2.set_index(drop=True,keys='movieId')

In [20]:
similarity_matrix2.head(5)

,key_0,1,2,3,4,5,6,7,8,9,...,193571,193573,193579,193581,193583,193585,193587,193609,title,genres
movieId,,,,,,,,,,,,,,,,,,,,,
1,0,1.000000,0.774597,0.316228,0.258199,0.447214,0.0,0.316228,0.632456,0.0,...,0.316228,0.447214,0.0,0.670820,0.774597,0.00000,0.316228,0.447214,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,1,0.774597,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.816497,0.0,...,0.000000,0.000000,0.0,0.288675,0.333333,0.00000,0.000000,0.000000,Jumanji (1995),Adventure|Children|Fantasy
3,2,0.316228,0.000000,1.000000,0.816497,0.707107,0.0,1.000000,0.000000,0.0,...,0.500000,0.000000,0.0,0.353553,0.408248,0.00000,0.000000,0.707107,Grumpier Old Men (1995),Comedy|Romance
4,3,0.258199,0.000000,0.816497,1.000000,0.577350,0.0,0.816497,0.000000,0.0,...,0.816497,0.000000,0.0,0.288675,0.333333,0.57735,0.000000,0.577350,Waiting to Exhale (1995),Comedy|Drama|Romance
5,4,0.447214,0.000000,0.707107,0.577350,1.000000,0.0,0.707107,0.000000,0.0,...,0.707107,0.000000,0.0,0.500000,0.577350,0.00000,0.000000,1.000000,Father of the Bride Part II (1995),Comedy


In [35]:
similarity_matrix3=similarity_matrix2.iloc[:,1:-2]

In [67]:
##Checking
similarity_matrix2[similarity_matrix2.index==174403]

,key_0,1,2,3,4,5,6,7,8,9,...,193571,193573,193579,193581,193583,193585,193587,193609,title,genres
movieId,,,,,,,,,,,,,,,,,,,,,
174403,9573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The Putin Interviews (2017),(no genres listed)


In [36]:
similarity_matrix3.head(3)

,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.774597,0.316228,0.258199,0.447214,0.0,0.316228,0.632456,0.0,0.258199,...,0.447214,0.316228,0.316228,0.447214,0.0,0.670820,0.774597,0.0,0.316228,0.447214
2,0.774597,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.816497,0.0,0.333333,...,0.000000,0.000000,0.000000,0.000000,0.0,0.288675,0.333333,0.0,0.000000,0.000000
3,0.316228,0.000000,1.000000,0.816497,0.707107,0.0,1.000000,0.000000,0.0,0.000000,...,0.353553,0.000000,0.500000,0.000000,0.0,0.353553,0.408248,0.0,0.000000,0.707107


In [37]:
pd.DataFrame(movies.movieId.unique())[0].drop_duplicates #Confirm no duplicate

<bound method Series.drop_duplicates of 0            1
1            2
2            3
3            4
4            5
         ...  
9737    193581
9738    193583
9739    193585
9740    193587
9741    193609
Name: 0, Length: 9742, dtype: int64>

In [110]:
## Prepare for function use.
from datetime import datetime
ratings2=ratings
ratings2['Date']=ratings2['timestamp'].apply(lambda x: datetime.fromtimestamp(x)) 
ratings2['Year']=ratings2['Date'].apply(lambda x: x.year)
ratings2['Month']=ratings2['Date'].apply(lambda x: x.month)
ratings2['Day']=ratings2['Date'].apply(lambda x: x.day)
#(ratings2.groupby(['movieId','Year']).agg({'userId':'count'})).sort_values('userId',ascending=False)

### Write a function to get movie id and return the top K most similar movies with
#### 1) min score, max score, 2)min rating, 3)min total rating, 4)time range

In [113]:
def backend_setting():
    TopK=int(input('How many movies need to pop out?: '))
    min_score=float(input('What is the min score?: ' ))
    max_score=float(input('What is the max score?: ' ))
    min_rating=float(input('What is the min rating?: ' ))
    min_total_rating=float(input('What is the min total rating?: ' ))
    time_start=input('What is the start time range (Format: YYYY-MM-DD e.g. 2020-11-03)?: ')
    time_end=input('What is the end time range (Format: YYYY-MM-DD e.g. 2020-11-04)?: ')
    return TopK,min_score,max_score,min_rating,min_total_rating,time_start,time_end

In [119]:
def frontend_setting():
    movie_select=int(input('Welcome! What movie do you want to watch today? Please give me a movieid: '))
    return movie_select

In [115]:
TopK,min_score,max_score,min_rating,min_total_rating,time_start,time_end=backend_setting()

How many movies need to pop out?:  5
What is the min score?:  0.5
What is the max score?:  0.8
What is the min rating?:  2
What is the min total rating?:  10
What is the start time range (Format: YYYY-MM-DD e.g. 2020-11-03)?:  1996-01-01
What is the end time range (Format: YYYY-MM-DD e.g. 2020-11-04)?:  1996-12-31


In [120]:
movie_select=frontend_setting()

Welcome! What movie do you want to watch today? Please give me a movieid:  592


In [121]:
print(TopK,min_score,max_score,min_rating,min_total_rating,time_start,time_end,movie_select)

5 0.5 0.8 2.0 10.0 1996-01-01 1996-12-31 592


In [122]:
def Recommend_Movie(TopK,min_score,max_score,min_rating,min_total_rating,time_start,time_end,movie_select):
    #movie_select=int(input('Welcome! What movie do you want to watch today? Please give me a movieid: '))
    r=movie_select
    k=TopK
    sort_c=pd.DataFrame(similarity_matrix3[r].sort_values(ascending=False)) ## Select the column with movie_select id
    sort_c2=pd.merge(sort_c,movies,left_on='movieId',right_on='movieId',how='inner') ## Join table to extract Movie Name
    sort_c3=pd.merge(sort_c2,ratings2,left_on='movieId',right_on='movieId',how='inner') ## Join table to extract Date and Ranking by Customer
    sort_c4=sort_c3[(sort_c3[r]>=min_score) & (sort_c3[r]<=max_score)
                    & (sort_c3['Date']>=datetime.strptime(time_start,'%Y-%m-%d'))
                    & (sort_c3['Date']<=datetime.strptime(time_end,'%Y-%m-%d'))
                    & (sort_c3['rating']>=min_rating)]
    sort_c5=(sort_c4.groupby(['movieId','title','genres']).agg({'userId':'count'})).sort_values('userId',ascending=False)
    sort_c6=sort_c5.reset_index()
    sort_c6=sort_c6.iloc[:k,:]
    sort_c6.columns=['Movie ID','Title','Genres','Total No. of Ranking']
    #sort_c2=sort_c[:k]
    #output1=pd.merge(sort_c2,movies,left_on=sort_c2.index,right_on='movieId',how='inner')
    #output1=output1.iloc[:,1:-1]
    print(sort_c6)

In [124]:
Recommend_Movie(TopK,min_score,max_score,min_rating,min_total_rating,time_start,time_end,movie_select)

   Movie ID                  Title                                    Genres  \
0       380       True Lies (1994)  Action|Adventure|Comedy|Romance|Thriller   
1       296    Pulp Fiction (1994)               Comedy|Crime|Drama|Thriller   
2       457   Fugitive, The (1993)                                  Thriller   
3       480   Jurassic Park (1993)          Action|Adventure|Sci-Fi|Thriller   
4       153  Batman Forever (1995)             Action|Adventure|Comedy|Crime   

   Total No. of Ranking  
0                    79  
1                    73  
2                    72  
3                    66  
4                    65  


### Reference

In [41]:
### Example to extract DateTime from TimeStamp
from datetime import datetime

timestamp = 964982703
dt_object = datetime.fromtimestamp(timestamp)

print("dt_object =", dt_object)
print("type(dt_object) =", type(dt_object))

dt_object.year

dt_object = 2000-07-31 02:45:03
type(dt_object) = <class 'datetime.datetime'>


2000